<a href="https://colab.research.google.com/github/ViMasHub/Test_Kontur/blob/main/Test-Kasperskiy-Colab(4)!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import re

import requests
from bs4 import BeautifulSoup

#Загрузка данных


In [ ]:
df_test = pd.read_excel('/content/Test_check.xlsx')
df_test.head()

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп..."


#Предобработка предоставленных данных


In [ ]:
# Установка библиотеки pymorphy3

!pip install pymorphy3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 54.1 MB/s eta 0:00:00


In [ ]:
# Импорт библиотек

import nltk
import pymorphy3
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Загружаем набор данных для токенизатора
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = stopwords.words('russian')
punctuation_marks = ['"','!', ',', '(', ')', ':', '-', '—', '?', '.', '..', '...', '»', '«']
morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
len(stop_words)

151

In [ ]:
# Функция предобработки тескта
def preprocess (text, stop_words, punctuation_marks, morph):

  no_num_text = re.sub(r'\d', '', text)

  tokens = word_tokenize(no_num_text.lower())

  preprocessed_text = []

  for token in tokens:

    if token not in punctuation_marks:

      lemma = morph.parse(token)[0].normal_form

      if lemma not in stop_words:

        preprocessed_text.append(lemma)

  return preprocessed_text

In [ ]:
df_test['pre_lib_text'] = df_test.apply(lambda row: preprocess(row['lib_text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци..."
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ..."


#Парсинг эталонов

In [ ]:
def get_data(url):

        fullData = requests.get(url, timeout=20)

        soup = BeautifulSoup(fullData.content, 'html.parser')

        for tag in soup.find_all(['script', 'style', 'noscript', 'iframe', 'header', 'footer', 'nav', 'aside']):

            tag.decompose()

        text_content = ' '

        domain = url.split('/')[2]

        if 'ria.ru' in domain:

            selectors = ['.article__text', '.layout-article__main']

        elif 'expert.ru' in domain:

            selectors = ['.article__text', '.article__content']

        elif 'lenta.ru' in domain:

            selectors = ['.__content-text', '.news-text']

        elif  'vesti.ru' in domain:

            selectors = ['js-mediator-article']

        else:
            selectors = ['.article', '.content', '.news-content', '.post-content', '.entry-content', 'main']

        # Находим текст по селекторам
        content_found = False
        for selector in selectors:
            content = soup.select(selector)
            if content:
                for element in content:
                    text_content += element.get_text() + ' '
                content_found = True
                break
        if not content_found:
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                text_content += p.get_text() + ' '

        text_content = re.sub(r'\s+', ' ', text_content).strip()

        return text_content

In [ ]:
df_test['ref_text'] = None

for i, url in enumerate(df_test['URL']):
        reference_text = get_data(url)
        df_test.at[i, 'ref_text'] = reference_text



In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text,ref_text,pre_ref_text,Percent_match
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",98.364008
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн...","МОСКВА, 25 апр — РИА Новости. Ключевая процент...","[москва, апр, риа, новость, ключевой, процентн...",94.459279
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",97.458810
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ...","МОСКВА, 4 мар - РИА Новости. Самозапрет на кре...","[москва, мара, риа, новость, самозапрет, креди...",95.481172
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...","Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...",97.996516


#Предобработка извлеченных данных

In [ ]:
df_test['pre_ref_text'] = df_test.apply(lambda row: preprocess(row['ref_text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
df_test.head()

,URL,lib_text,pre_lib_text,ref_text,pre_ref_text,Percent_match
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",98.364008
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн...","МОСКВА, 25 апр — РИА Новости. Ключевая процент...","[москва, апр, риа, новость, ключевой, процентн...",94.459279
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",97.458810
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ...","МОСКВА, 4 мар - РИА Новости. Самозапрет на кре...","[москва, мара, риа, новость, самозапрет, креди...",95.481172
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...","Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...",97.996516


# Определение полноты извлеченного текста

In [ ]:
# Импорт библиотеки
import difflib

# Функция сравнения текстов
def percent_similarity(text1, text2):

    seq = difflib.SequenceMatcher(None, text1, text2)

    if seq.ratio() * 100 < 75:

      print(seq.ratio() * 100)

    return seq.ratio() * 100  # возвращает в процентах

In [ ]:
df_test['Percent_match'] = df_test.apply(lambda row: percent_similarity(row['pre_lib_text'], row['pre_ref_text']), axis=1)

63.84039900249376


In [ ]:
# По одной сылке тексты обладают средним сходством (50-74%), по остальным высоким (75-89%) или очень высоким (90-100%) сходством.

In [ ]:
df_test

,URL,lib_text,pre_lib_text,ref_text,pre_ref_text,Percent_match
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци...",98.364008
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн...","МОСКВА, 25 апр — РИА Новости. Ключевая процент...","[москва, апр, риа, новость, ключевой, процентн...",94.459279
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро...",97.458810
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ...","МОСКВА, 4 мар - РИА Новости. Самозапрет на кре...","[москва, мара, риа, новость, самозапрет, креди...",95.481172
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...","Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ...",97.996516
5,https://lenta.ru/articles/2025/03/09/aslan/,"20 лет назад, 8 марта 2005 года, в чеченском с...","[год, назад, март, год, чеченский, село, толст...",Фото: Машатин Владимир / PhotoXPress 20 лет на...,"[фото, машатин, владимир, /, photoxpress, год,...",88.257466
6,https://expert.ru/finance/investorov-svyazyvay...,"Говоря сухим языком определений, цифровые фина...","[говорить, сухой, язык, определение, цифровой,...","Говоря сухим языком определений, цифровые фина...","[говорить, сухой, язык, определение, цифровой,...",97.010608
7,https://ria.ru/20250123/svo-1985822676.html,Поддержка военнослужащих и их семей с самого н...,"[поддержка, военнослужащий, семья, сам, начало...","МОСКВА, 23 янв - РИА Новости. Выплаты участник...","[москва, янв, риа, новость, выплата, участник,...",93.463374
8,https://expert.ru/mnenie/vse-teper-zavisit-tol...,— За последние два с половиной года из-за введ...,"[последний, половина, год, из-за, ввести, санк...",— За последние два с половиной года из-за введ...,"[последний, половина, год, из-за, ввести, санк...",95.723492
9,https://ria.ru/docs/about/privacy_policy.html,Федеральное государственное унитарное предприя...,"[федеральный, государственный, унитарный, пред...",Федеральное государственное унитарное предприя...,"[федеральный, государственный, унитарный, пред...",83.126883


Оценка значимости пропущенного текста
